# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [321]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Ask for parameters

In [322]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [323]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

*  **OF**: Olefin feed (barrels per day)
*  **IBR**: Isobutane recycle (barrels per day)
*  **AAR**: Acid addition rate (thousands of pounds per day)
*  **AY**: Alkylate yield (barrels per day)
*  **IBM**: Isobutane makeup (barrels per day)
*  **AS**: Acid strength (weight percent)
*  **MON**: Motor octane number
*  **IOR**: External isobutane-to-olefin ratio
*  **ADF**: Acid dilution factor
*  **F4**: F-4 performance number 


# 2. Ask for objective

In [324]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [325]:
Markdown(response2.text)

## Objective Function:

Maximize **Daily Profit (DP)**

**DP** = (**AY** * **MON** * **Revenue Per Barrel**) - (**OF** * **Olefin Cost Per Barrel**) - (**IBR** * **Isobutane Cost Per Barrel**) - (**AAR** * **Acid Cost Per Pound** / 1000) - (**IBM** * **Isobutane Cost Per Barrel**) 

**Where:**

* **Revenue Per Barrel**: Revenue generated per barrel of alkylate.
* **Olefin Cost Per Barrel**: Cost per barrel of olefin feed.
* **Isobutane Cost Per Barrel**: Cost per barrel of isobutane.
* **Acid Cost Per Pound**: Cost per pound of acid. 


# 3. Ask for constraints

In [326]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [327]:
Markdown(response3.text)

## Constraints:

**Variable Bounds:**

* **OF_lowerbound** ≤ **OF** ≤ **OF_upperbound**
* **IBR_lowerbound** ≤ **IBR** ≤ **IBR_upperbound**
* **AAR_lowerbound** ≤ **AAR** ≤ **AAR_upperbound**
* **AY_lowerbound** ≤ **AY** ≤ **AY_upperbound**
* **IBM_lowerbound** ≤ **IBM** ≤ **IBM_upperbound**
* **AS_lowerbound** ≤ **AS** ≤ **AS_upperbound**
* **MON_lowerbound** ≤ **MON** ≤ **MON_upperbound**
* **IOR_lowerbound** ≤ **IOR** ≤ **IOR_upperbound**
* **ADF_lowerbound** ≤ **ADF** ≤ **ADF_upperbound**
* **F4_lowerbound** ≤ **F4** ≤ **F4_upperbound**

**Regression Formula Deviations:**

* **0.98 * AY_calculated** ≤ **AY** ≤ **1.02 * AY_calculated** 
* **0.99 * MON_calculated** ≤ **MON** ≤ **1.01 * MON_calculated** 
* **0.95 * ADF_calculated** ≤ **ADF** ≤ **1.05 * ADF_calculated** 
* **0.9 * F4_calculated** ≤ **F4** ≤ **1.1 * F4_calculated** 

**Where:**

* **AY_calculated  = OF * (1.12 + 0.13167 * IOR - 0.00667 * IOR^2)**
* **MON_calculated = 86.35 + 1.098 * IOR - 0.038 * IOR^2 + 0.325 * (AS - 89)**
* **ADF_calculated = 35.82 - 0.222 * F4**
* **F4_calculated = -133 + 3 * MON**

**Operational Constraints:**

* **IOR = (IBR + IBM) / OF**
* **AS = (93000 * AAR) / (AAR * ADF + 1000 * AAR)**
* **1.22 * AY =  OF + IBM** 


## 3. Generate the pyomo code

In [328]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [329]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
revenue_per_barrel = 100  
olefin_cost_per_barrel = 50 
isobutane_cost_per_barrel = 40
acid_cost_per_pound = 2

# Variable bounds (replace with actual bounds)
bounds = {
    'OF': (5000, 10000),
    'IBR': (1000, 5000),
    'AAR': (10, 50), 
    'AY': (1000, 10000),
    'IBM': (500, 2000),
    'AS': (88, 92), 
    'MON': (88, 94),
    'IOR': (3, 6), 
    'ADF': (25, 35), 
    'F4': (130, 140)
}

# Model
model = pyo.ConcreteModel()

# Variables
model.OF = pyo.Var(bounds=bounds['OF'])
model.IBR = pyo.Var(bounds=bounds['IBR'])
model.AAR = pyo.Var(bounds=bounds['AAR'])
model.AY = pyo.Var(bounds=bounds['AY'])
model.IBM = pyo.Var(bounds=bounds['IBM'])
model.AS = pyo.Var(bounds=bounds['AS'])
model.MON = pyo.Var(bounds=bounds['MON'])
model.IOR = pyo.Var(bounds=bounds['IOR'])
model.ADF = pyo.Var(bounds=bounds['ADF'])
model.F4 = pyo.Var(bounds=bounds['F4'])

# Objective function
def daily_profit(model):
    return (model.AY * model.MON * revenue_per_barrel 
            - model.OF * olefin_cost_per_barrel 
            - model.IBR * isobutane_cost_per_barrel
            - model.AAR * acid_cost_per_pound / 1000
            - model.IBM * isobutane_cost_per_barrel)
model.DP = pyo.Objective(rule=daily_profit, sense=pyo.maximize)

# Constraints
model.AY_calc = pyo.Constraint(expr=0.98 * model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) <= model.AY)
model.AY_calc_upper = pyo.Constraint(expr= model.AY <= 1.02 * model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2))
model.MON_calc = pyo.Constraint(expr= 0.99 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89)) <= model.MON)
model.MON_calc_upper = pyo.Constraint(expr=model.MON <= 1.01 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89)))
model.ADF_calc = pyo.Constraint(expr=0.95 * (35.82 - 0.222 * model.F4) <= model.ADF)
model.ADF_calc_upper = pyo.Constraint(expr=model.ADF <= 1.05 * (35.82 - 0.222 * model.F4))
model.F4_calc = pyo.Constraint(expr= 0.9 * (-133 + 3 * model.MON) <= model.F4)
model.F4_calc_upper = pyo.Constraint(expr= model.F4 <= 1.1 * (-133 + 3 * model.MON))

model.IOR_def = pyo.Constraint(expr= model.IOR == (model.IBR + model.IBM) / model.OF)
model.AS_def = pyo.Constraint(expr= model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))
model.Alkylate_balance = pyo.Constraint(expr= 1.22 * model.AY == model.OF + model.IBM)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver
results = solver.solve(model)

# Print the results
print(results)
print("Daily Profit:", pyo.value(model.DP)) 
print("Olefin Feed:", pyo.value(model.OF))
# ... print other variable values

```

**Explanation:**

1. **Import Pyomo:**  Begin by importing the Pyomo library.
2. **Sample Data:**  Define the sample data for costs and revenue.  You'll want to replace these with your actual values.
3. **Variable Bounds:** Define the upper and lower bounds for each variable. This helps to constrain the search space for the solver.
4. **Model:** Create a ConcreteModel in Pyomo to hold the optimization problem.
5. **Variables:** Define each variable within your model (OF, IBR, AAR, etc.) and associate them with their respective bounds. 
6. **Objective Function:** Define the objective function `daily_profit` to calculate the profit based on the model variables and input data.
7. **Constraints:** Implement all the constraints, including the regression formula deviations and operational constraints.
8. **Solver:** Select a solver (in this case, IPOPT) and apply it to your Pyomo model.
9. **Results:** Display the optimization results, including the optimal daily profit and the values of the decision variables that achieve this maximum profit.

Remember that the provided data is for illustration purposes.  Replace the sample data, bounds, and potentially the solver with ones suitable for your specific alkylation process model. 


## 4. Input problem data and try running the generated code

In [330]:
import pyomo.environ as pyo

# Costs
c1, c2, c3, c4, c5 = 0.63, 5.04, 0.035, 10.0, 3.36

# Lower and upper bounds
x1_l, x1_u = (0, 2000)
x2_l, x2_u = (0, 16000)
x3_l, x3_u = (0, 120)
x4_l, x4_u = (0, 5000)
x5_l, x5_u = (0, 2000)
x6_l, x6_u = (85, 93)
x7_l, x7_u = (90, 95)
x8_l, x8_u = (3, 12)
x9_l, x9_u = (1.2, 4)
x10_l, x10_u = (145, 162)

# Decision Variables:
#     x1 (continuous): Olefin feed (barrels per day)
#     x2 (continuous): Isobutane recycle (barrels per day)
#     x3 (continuous): Acid addition rate (thousands of pounds per day)
#     x4 (continuous): Alkylate yield (barrels per day)
#     x5 (continuous): Isobutane makeup (barrels per day)
#     x6 (continuous): Acid strength (weight percent)
#     x7 (continuous): Motor octane number
#     x8 (continuous): External isobutane-to-olefin ratio
#     x9 (continuous): Acid dilution factor
#     x10 (continuous): F-4 performance number

# Parameters:
#     c1 to c5: Costs (used in the objective function)
#     x_i^l: Lower bound of variable i
#     x_i^u: Upper bound of variable i
# Sample Data (replace with actual data if available)
data = {
    'ε_Alk': 0.63,  # Sample revenue per barrel of alkylate (replace with actual pricing)
    'C_o': 5.04,     # Sample cost per barrel of olefin feed
    'C_i': 0.035,     # Sample cost per barrel of isobutane
    'C_aA': 10,   # Sample cost per thousand pounds of acid addition
    'F_o_min': 0,  # Sample minimum olefin feed
    'F_o_max': 2000,  # Sample maximum olefin feed
    'F_iR_min': 0,   # Sample minimum isobutane recycle
    'F_iR_max': 16000,  # Sample maximum isobutane recycle
    'F_aA_min': 0,     # Sample minimum acid addition rate
    'F_aA_max': 120,    # Sample maximum acid addition rate
    'F_iM_min': 0,   # Sample minimum isobutane makeup
    'F_iM_max': 5000,  # Sample maximum isobutane makeup
}

#Previous is for data copying, here is the model

import pyomo.environ as pyo

# Sample data (replace with actual data)
revenue_per_barrel = 100  
olefin_cost_per_barrel = 50 
isobutane_cost_per_barrel = 40
acid_cost_per_pound = 2

# Variable bounds (replace with actual bounds)
bounds = {
    'OF': (5000, 10000),
    'IBR': (1000, 5000),
    'AAR': (10, 50), 
    'AY': (1000, 10000),
    'IBM': (500, 2000),
    'AS': (88, 92), 
    'MON': (88, 94),
    'IOR': (3, 6), 
    'ADF': (25, 35), 
    'F4': (130, 140)
}

# Model
model = pyo.ConcreteModel()

# Variables
model.OF = pyo.Var(bounds=bounds['OF'])
model.IBR = pyo.Var(bounds=bounds['IBR'])
model.AAR = pyo.Var(bounds=bounds['AAR'])
model.AY = pyo.Var(bounds=bounds['AY'])
model.IBM = pyo.Var(bounds=bounds['IBM'])
model.AS = pyo.Var(bounds=bounds['AS'])
model.MON = pyo.Var(bounds=bounds['MON'])
model.IOR = pyo.Var(bounds=bounds['IOR'])
model.ADF = pyo.Var(bounds=bounds['ADF'])
model.F4 = pyo.Var(bounds=bounds['F4'])

# Objective function
def daily_profit(model):
    return (model.AY * model.MON * revenue_per_barrel 
            - model.OF * olefin_cost_per_barrel 
            - model.IBR * isobutane_cost_per_barrel
            - model.AAR * acid_cost_per_pound / 1000
            - model.IBM * isobutane_cost_per_barrel)
model.DP = pyo.Objective(rule=daily_profit, sense=pyo.maximize)

# Constraints
model.AY_calc = pyo.Constraint(expr=0.98 * model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) <= model.AY)
model.AY_calc_upper = pyo.Constraint(expr= model.AY <= 1.02 * model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2))
model.MON_calc = pyo.Constraint(expr= 0.99 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89)) <= model.MON)
model.MON_calc_upper = pyo.Constraint(expr=model.MON <= 1.01 * (86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89)))
model.ADF_calc = pyo.Constraint(expr=0.95 * (35.82 - 0.222 * model.F4) <= model.ADF)
model.ADF_calc_upper = pyo.Constraint(expr=model.ADF <= 1.05 * (35.82 - 0.222 * model.F4))
model.F4_calc = pyo.Constraint(expr= 0.9 * (-133 + 3 * model.MON) <= model.F4)
model.F4_calc_upper = pyo.Constraint(expr= model.F4 <= 1.1 * (-133 + 3 * model.MON))

model.IOR_def = pyo.Constraint(expr= model.IOR == (model.IBR + model.IBM) / model.OF)
model.AS_def = pyo.Constraint(expr= model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))
model.Alkylate_balance = pyo.Constraint(expr= 1.22 * model.AY == model.OF + model.IBM)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver
results = solver.solve(model)

# Print the results
print(results)
print("Daily Profit:", pyo.value(model.DP)) 
print("Olefin Feed:", pyo.value(model.OF))
# ... print other variable values

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.11.1\x3a Converged to a locally infeasible
      point. Problem may be infeasible.

Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 10
  Sense: unknown
Solver: 
- Status: warning
  Message: Ipopt 3.11.1\x3a Converged to a locally infeasible point. Problem may be infeasible.
  Termination condition: infeasible
  Id: 200
  Error rc: 0
  Time: 0.08198356628417969
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Daily Profit: 51175998.20147992
Olefin Feed: 5000.0


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [331]:
print(response.text)

## Variables:

*  **OF**: Olefin feed (barrels per day)
*  **IBR**: Isobutane recycle (barrels per day)
*  **AAR**: Acid addition rate (thousands of pounds per day)
*  **AY**: Alkylate yield (barrels per day)
*  **IBM**: Isobutane makeup (barrels per day)
*  **AS**: Acid strength (weight percent)
*  **MON**: Motor octane number
*  **IOR**: External isobutane-to-olefin ratio
*  **ADF**: Acid dilution factor
*  **F4**: F-4 performance number 



In [332]:
print(response2.text)

## Objective Function:

Maximize **Daily Profit (DP)**

**DP** = (**AY** * **MON** * **Revenue Per Barrel**) - (**OF** * **Olefin Cost Per Barrel**) - (**IBR** * **Isobutane Cost Per Barrel**) - (**AAR** * **Acid Cost Per Pound** / 1000) - (**IBM** * **Isobutane Cost Per Barrel**) 

**Where:**

* **Revenue Per Barrel**: Revenue generated per barrel of alkylate.
* **Olefin Cost Per Barrel**: Cost per barrel of olefin feed.
* **Isobutane Cost Per Barrel**: Cost per barrel of isobutane.
* **Acid Cost Per Pound**: Cost per pound of acid. 



In [333]:
print(response3.text)

## Constraints:

**Variable Bounds:**

* **OF_lowerbound** ≤ **OF** ≤ **OF_upperbound**
* **IBR_lowerbound** ≤ **IBR** ≤ **IBR_upperbound**
* **AAR_lowerbound** ≤ **AAR** ≤ **AAR_upperbound**
* **AY_lowerbound** ≤ **AY** ≤ **AY_upperbound**
* **IBM_lowerbound** ≤ **IBM** ≤ **IBM_upperbound**
* **AS_lowerbound** ≤ **AS** ≤ **AS_upperbound**
* **MON_lowerbound** ≤ **MON** ≤ **MON_upperbound**
* **IOR_lowerbound** ≤ **IOR** ≤ **IOR_upperbound**
* **ADF_lowerbound** ≤ **ADF** ≤ **ADF_upperbound**
* **F4_lowerbound** ≤ **F4** ≤ **F4_upperbound**

**Regression Formula Deviations:**

* **0.98 * AY_calculated** ≤ **AY** ≤ **1.02 * AY_calculated** 
* **0.99 * MON_calculated** ≤ **MON** ≤ **1.01 * MON_calculated** 
* **0.95 * ADF_calculated** ≤ **ADF** ≤ **1.05 * ADF_calculated** 
* **0.9 * F4_calculated** ≤ **F4** ≤ **1.1 * F4_calculated** 

**Where:**

* **AY_calculated  = OF * (1.12 + 0.13167 * IOR - 0.00667 * IOR^2)**
* **MON_calculated = 86.35 + 1.098 * IOR - 0.038 * IOR^2 + 0.325 * (AS - 8

In [334]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
revenue_per_barrel = 100  
olefin_cost_per_barrel = 50 
isobutane_cost_per_barrel = 40
acid_cost_per_pound = 2

# Variable bounds (replace with actual bounds)
bounds = {
    'OF': (5000, 10000),
    'IBR': (1000, 5000),
    'AAR': (10, 50), 
    'AY': (1000, 10000),
    'IBM': (500, 2000),
    'AS': (88, 92), 
    'MON': (88, 94),
    'IOR': (3, 6), 
    'ADF': (25, 35), 
    'F4': (130, 140)
}

# Model
model = pyo.ConcreteModel()

# Variables
model.OF = pyo.Var(bounds=bounds['OF'])
model.IBR = pyo.Var(bounds=bounds['IBR'])
model.AAR = pyo.Var(bounds=bounds['AAR'])
model.AY = pyo.Var(bounds=bounds['AY'])
model.IBM = pyo.Var(bounds=bounds['IBM'])
model.AS = pyo.Var(bounds=bounds['AS'])
model.MON = pyo.Var(bounds=bounds['MON'])
model.IOR = pyo.Var(bounds=bounds['IOR'])
model.ADF = pyo.Var(bounds=bounds['ADF'])
model.F4 = pyo.Var(bounds=bounds['F4'])

# Objective function
def daily_profit(model):
    return (m